# Let's prototype some python analysis and logic

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('game_results.csv')

In [ ]:
data

In [3]:
# Convert datetime to pandas datetime
data['datetime'] = pd.to_datetime(data['datetime'])

# Get min and max dates
min_date = data['datetime'].min().date()
max_date = data['datetime'].max().date()

# Create date range including all days
date_range = pd.date_range(start=min_date, end=max_date, freq='D')

# Count games per day
games_per_day = data.groupby(data['datetime'].dt.date).size()

# Reindex to include all dates, filling missing dates with 0
games_per_day = games_per_day.reindex(date_range.date, fill_value=0)


In [ ]:
# Create bar plot
plt.style.use("fivethirtyeight")
plt.figure(figsize=(15,6))
plt.bar(games_per_day.index, games_per_day.values)
plt.title('Number of Games Played per Day')
plt.xlabel('Date')
plt.ylabel('Number of Games')
plt.tight_layout()
plt.margins(x=0.01)
plt.show()


In [ ]:
# Plot win rates by character
plt.style.use("fivethirtyeight")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# Define Gruvbox colors
orange = '#fe8019'  # Shayne's color
green = '#b8bb26'   # Matt's color
bg = '#282828'      # Background
fg = '#ebdbb2'      # Foreground/text
blue = '#83a598'    # For other plots

# Character win rates for Shayne
shayne_char_stats = data[data['winner'] == 'Shayne'].groupby('shayne_character').size()
shayne_total_games = data.groupby('shayne_character').size()
shayne_winrates = (shayne_char_stats / shayne_total_games * 100).sort_values(ascending=False)
shayne_winrates = shayne_winrates[shayne_total_games[shayne_winrates.index] >= 10]  # Filter for chars with 10+ games

bars1 = ax1.bar(shayne_winrates.index, shayne_winrates.values, color=orange)
ax1.set_title("Shayne's Character Win Rates (10+ games)", color=fg)
ax1.set_ylabel("Win Rate (%)", color=fg)
ax1.tick_params(axis='x', rotation=45, colors=fg)
ax1.tick_params(axis='y', colors=fg)
ax1.set_ylim(0, 80)
# Add win rate labels on top of bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.0f}%', ha='center', va='bottom', color=fg, fontweight='bold')

# Character win rates for Matt
matt_char_stats = data[data['winner'] == 'Matt'].groupby('matt_character').size()
matt_total_games = data.groupby('matt_character').size()
matt_winrates = (matt_char_stats / matt_total_games * 100).sort_values(ascending=False)
matt_winrates = matt_winrates[matt_total_games[matt_winrates.index] >= 10]  # Filter for chars with 10+ games

bars2 = ax2.bar(matt_winrates.index, matt_winrates.values, color=green)
ax2.set_title("Matt's Character Win Rates (10+ games)", color=fg)
ax2.set_ylabel("Win Rate (%)", color=fg)
ax2.tick_params(axis='x', rotation=45, colors=fg)
ax2.tick_params(axis='y', colors=fg)
ax2.set_ylim(0, 80)
# Add win rate labels on top of bars
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.0f}%', ha='center', va='bottom', color=fg, fontweight='bold')

# Most played stages (excluding "No Stage")
stage_data = data[data['stage'] != 'No Stage']
stage_counts = stage_data['stage'].value_counts()
bars3 = ax3.bar(stage_counts.index, stage_counts.values, color=blue)
ax3.set_title("Stage Selection Frequency", color=fg)
ax3.set_ylabel("Number of Games", color=fg)
ax3.tick_params(axis='x', rotation=45, colors=fg)
ax3.tick_params(axis='y', colors=fg)
# Add count labels inside bars
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height/2,
             f'{int(height)}', ha='center', va='center', color=fg, fontweight='bold')

# Stock distribution by player
stock_data_shayne = data[data['winner'] == 'Shayne']['stocks_remaining'].value_counts().sort_index()
stock_data_matt = data[data['winner'] == 'Matt']['stocks_remaining'].value_counts().sort_index()

# Ensure all stock values are present for both players
all_stocks = range(1, max(data['stocks_remaining']) + 1)
stock_data_shayne = stock_data_shayne.reindex(all_stocks, fill_value=0)
stock_data_matt = stock_data_matt.reindex(all_stocks, fill_value=0)

bars4_matt = ax4.bar(stock_data_matt.index, stock_data_matt.values, color=green, label="Matt")
bars4_shayne = ax4.bar(stock_data_shayne.index, stock_data_shayne.values, bottom=stock_data_matt.values, 
        color=orange, label="Shayne")
ax4.set_title("Distribution of Stocks Remaining by Winner", color=fg)
ax4.set_xlabel("Stocks Remaining", color=fg)
ax4.set_ylabel("Number of Games", color=fg)
ax4.tick_params(axis='x', colors=fg)
ax4.tick_params(axis='y', colors=fg)
ax4.set_xticks(all_stocks)
ax4.legend(facecolor=bg, labelcolor=fg)

# Add count labels inside stacked bars
for i, (matt_bar, shayne_bar) in enumerate(zip(bars4_matt, bars4_shayne)):
    matt_height = matt_bar.get_height()
    shayne_height = shayne_bar.get_height()
    
    # Matt's labels (bottom bars)
    if matt_height > 0:
        ax4.text(matt_bar.get_x() + matt_bar.get_width()/2., matt_height/2,
                f'{int(matt_height)}', ha='center', va='center', color=fg, fontweight='bold')
    
    # Shayne's labels (top bars)
    if shayne_height > 0:
        ax4.text(shayne_bar.get_x() + shayne_bar.get_width()/2., matt_height + shayne_height/2,
                f'{int(shayne_height)}', ha='center', va='center', color=fg, fontweight='bold')

# Set figure background color
fig.patch.set_facecolor(bg)
for ax in [ax1, ax2, ax3, ax4]:
    ax.set_facecolor(bg)

plt.tight_layout()
plt.show()

# Print overall win rates
total_games = len(data)
shayne_wins = len(data[data['winner'] == 'Shayne'])
matt_wins = len(data[data['winner'] == 'Matt'])

print(f"\nOverall Statistics:")
print(f"Total Games: {total_games}")
print(f"Shayne's Win Rate: {shayne_wins/total_games*100:.1f}%")
print(f"Matt's Win Rate: {matt_wins/total_games*100:.1f}%")
